In [1]:
import multiprocessing as mp

mp.cpu_count()

12

In [31]:
def convert_filename(filename):
    parts = filename.split("_")
    directory = "_".join(parts[:-1])
    new_filename = f"{directory}/{parts[-1]}"
    return new_filename

In [41]:
#클래스 별 랜덤하게 크롭해서 저장
import os
import json
import cv2
import random
import shutil
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed

def crop_this_shit(annotation, category_folder, imgs, category_name):
    
    if annotation['area'] < 0:
        return {}

    filename = None
    for img in imgs:
        if img['id'] == annotation['image_id']:
            filename = img['file_name']
            #IR data 가 들어올 경우
            filename = convert_filename(filename)

    assert filename != None, "Filename not defined"
    
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)

    # bounding box 추출 및 crop
    bbox = annotation['bbox']
    xmin, ymin, width, height = map(int, bbox)
    xmax, ymax = xmin + width, ymin + height
    cropped_image = image[ymin:ymax, xmin:xmax]

    # crop된 이미지 저장
    annotation_id = "annotationId_" + str(annotation['id']) + "." + filename.split('.')[-1]
    save_filename = f"crop_{annotation_id}"
    save_path = os.path.join(category_folder, save_filename)
    # print("start!")
    # print("id : " + str(annotation['id']))
    # print("category : " + str(category_name))
    # print("filename : " + filename)
    try:
        cv2.imwrite(save_path, cropped_image)
    except:
        return {
            
        }
    is_valid = True if "test" or "val" in image_folder else False

    return {
        'path': save_path,
        'label': str(category_name),
        'is_valid': is_valid,
        'width': width,
        'height': height
    }

def crop_and_save_images(image_folder, annotations_path, dest_folder, num_samples=3):
    # COCO 형식의 어노테이션 파일 로드
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)

    img = annotations['images']
    print(img[0])
    
    # 클래스별 이미지 목록 생성
    class_images = {}
    for annotation in annotations['annotations']:
        category_id = annotation['category_id']
        category_name = None
        for category in annotations['categories']:
            if category['id'] == category_id:
                category_name = category['id']
                break
        if category_name != None:
            class_images.setdefault(category_name, []).append(annotation)

    data = []
    # 클래스별로 랜덤하게 이미지 샘플링하여 crop 및 저장
    for category_name, annotation_list in class_images.items():
        selected_annotations = random.sample(annotation_list, min(num_samples, len(annotation_list)))
        category_name = "{:02d}".format(category_name)
        category_folder = os.path.join(dest_folder, str(category_name))
        os.makedirs(category_folder, exist_ok=True)

        with Parallel(n_jobs=mp.cpu_count()) as parallel:
            data = parallel(delayed(crop_this_shit)(annotation, category_folder, img, category_name) for annotation in tqdm(selected_annotations))
            data = [d for d in data if d]
    # DataFrame 생성
    df = pd.DataFrame(data)
    return df

# 이미지 폴더 경로 및 어노테이션 파일 경로 설정
image_folder = '/home/ubuntu/workspace/datasets/final_dataset/train/IR/image'
annotations_path = '/home/ubuntu/workspace/datasets/final_dataset/train/annotations/IR-val.json'
dest_folder = '/home/ubuntu/workspace/visualization/crop/IR_val'

# 클래스별 샘플링 이미지 수 설정 (기본값: 3)
num_samples_per_class = 500

# crop과 이미지 저장 수행
df = crop_and_save_images(image_folder, annotations_path, dest_folder, num_samples=num_samples_per_class)

{'id': 0, 'file_name': '20190926_193515_1_8_000001.jpg', 'license': 0, 'width': 640, 'height': 512}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:03<00:00, 154.96it/s]


In [7]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [ ]:
# 모든 파일 class 별로 visualize
import os
import json
import cv2
import random
import shutil

def crop_and_save_images_by_classes(image_folder, annotations_path, dest_folder):
    # COCO 형식의 어노테이션 파일 로드
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)

    imgs = annotations['images']
    
    # 클래스별 이미지 목록 생성
    class_images = {}
    for annotation in annotations['annotations']:
        category_id = annotation['category_id']
        category_name = None
        for category in annotations['categories']:
            if category['id'] == category_id:
                category_name = category['name']
                break
        if category_name:
            class_images.setdefault(category_name, []).append(annotation)

    # 클래스별로 랜덤하게 이미지 샘플링하여 crop 및 저장
    for category_name, annotation_list in class_images.items():
        selected_annotations = annotation_list
        category_folder = os.path.join(dest_folder, category_name)
        os.makedirs(category_folder, exist_ok=True)

        for annotation in selected_annotations:
            if annotation['area'] < 0:
                continue
            print("start!")
            print("id : " + str(annotation['id']))
            
            for img in imgs:
                if img['id'] == annotation['image_id']:
                    filename = img['file_name']
                    break
                
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)

            # bounding box 추출 및 crop
            bbox = annotation['bbox']
            xmin, ymin, width, height = map(int, bbox)
            xmax, ymax = xmin + width, ymin + height
            cropped_image = image[ymin:ymax, xmin:xmax]

            # crop된 이미지 저장
            save_filename = f"crop_{filename}"
            save_path = os.path.join(category_folder, save_filename)
            print("category : " + category_name)
            print("filename : " + filename)
            cv2.imwrite(save_path, cropped_image)
            print(filename + " end!")

# 이미지 폴더 경로 및 어노테이션 파일 경로 설정
image_folder = '/home/jupyter/data/images/'
annotations_path = '/home/jupyter/data/sample_by_classes.json'
dest_folder = '/home/jupyter/data/crop_image_by_class'

# crop과 이미지 저장 수행
crop_and_save_images_by_classes(image_folder, annotations_path, dest_folder)

In [ ]:
#하나 visualize
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np

def crop_and_visualize(image_path, annotations_path):
    # 이미지 로드
    image = cv2.imread(image_path)
    file_name = image_path.split('/')[-1]
    
    # COCO 형식의 어노테이션 파일 로드
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    
    img = annotations['images']
    
    # bounding box 추출 및 crop
    for annotation in annotations['annotations']:
        if file_name == img[annotation['image_id']]['file_name']:
            bbox = annotation['bbox']
            xmin, ymin, width, height = map(int, bbox)
            xmax, ymax = xmin + width, ymin + height
            cropped_image = image[ymin:ymax, xmin:xmax]

            # crop된 이미지 시각화
            plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()

# 이미지 파일 경로 및 어노테이션 파일 경로 설정
image_path = '/home/jupyter/data/images/064507368.jpg'
annotations_path = '/home/jupyter/data/prediction.json'

# crop과 visualize 수행
crop_and_visualize(image_path, annotations_path)

In [ ]:
#폴더 전체 visualize
import os
import json
import cv2
import matplotlib.pyplot as plt

def crop_and_visualize_folder_images(image_folder, annotations_path):
    # COCO 형식의 어노테이션 파일 로드
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
        
    img = annotations['images']
    
    # 이미지 폴더 내의 모든 이미지에 대해 crop 및 visualize
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)

            # bounding box 추출 및 crop
            for annotation in annotations['annotations']:
                if filename == img[annotation['image_id']]['file_name']:
                    bbox = annotation['bbox']
                    xmin, ymin, width, height = map(int, bbox)
                    xmax, ymax = xmin + width, ymin + height
                    cropped_image = image[ymin:ymax, xmin:xmax]

                    # crop된 이미지 시각화
                    plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                    plt.axis('off')
                    plt.show()

# 이미지 폴더 경로 및 어노테이션 파일 경로 설정
image_folder = '/home/jupyter/data/images'
annotations_path = '/home/jupyter/data/prediction.json'

# crop과 visualize 수행
crop_and_visualize_folder_images(image_folder, annotations_path)

In [33]:
%cd /home/ubuntu/workspace/visualization/crop

/home/ubuntu/workspace/visualization/crop


In [37]:
!rm -r EO_train